In [1]:
# Reload automatically when the file is changed.
%load_ext autoreload
%autoreload 2

# Importing Modules

In [2]:
import os 
import sys
MODULES_PATH = "../../Modules/Uni_Model/"
MODELS_PATH = "../../Models"

sys.path.append(MODULES_PATH)
sys.path.append(MODELS_PATH)


from Preprocessing import *
from Dataloaders_Preprocessing import *
from Train import *
from Uni_Model import *






# Importing Data

In [3]:
List_Data_Train = Load_Data('../../Data/atoms/train', '../../Data/energies/train.csv')
List_Data_Test= Load_Test_Data('../../Data/atoms/test')

In [4]:
Display_Molecule_From_Atom_List(List_Data_Train[0]['Atoms_List'], Width=800, Height=800, Background_Color='lightblue')


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:

Dataset_Train = MoleculeDataset_Uni_Model(Data_List = List_Data_Train,
                                             Return_Energies=True)

Dataset_Test = MoleculeDataset_Uni_Model(Data_List = List_Data_Test,
                                             Return_Energies=False)




In [6]:
Dataset_Train[0]

(5415,
 array(['[CLS]', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'O', 'H', 'H', 'H'],
       dtype=object),
 tensor([6, 1, 1, 1, 1, 0, 0, 0, 3, 0, 0, 0]),
 tensor([[-0.2947,  0.3089, -0.1524],
         [ 0.0766, -0.5044,  0.2825],
         [ 0.8097,  0.6040, -0.4141],
         [-1.4463, -0.4365,  0.3707],
         [ 0.2336,  1.6711, -0.9761],
         [-0.8422,  1.8130, -0.9669],
         [-1.8898, -0.4044, -0.6233],
         [-1.7641,  0.4604,  0.9005],
         [ 0.6962, -1.4385,  0.7658],
         [ 1.8935,  0.5000, -0.4464],
         [-1.8323, -1.3050,  0.8957],
         [ 0.8234,  2.4386, -1.4647]]),
 tensor([[48, 43, 43, 43, 43, 42, 42, 42, 45, 42, 42, 42],
         [13,  8,  8,  8,  8,  7,  7,  7, 10,  7,  7,  7],
         [13,  8,  8,  8,  8,  7,  7,  7, 10,  7,  7,  7],
         [13,  8,  8,  8,  8,  7,  7,  7, 10,  7,  7,  7],
         [13,  8,  8,  8,  8,  7,  7,  7, 10,  7,  7,  7],
         [ 6,  1,  1,  1,  1,  0,  0,  0,  3,  0,  0,  0],
         [ 6,  1,  1,  1,  1,  0,  0,  

In [7]:
# Configuration des paramètres pour la classe Uni_Model
NUM_LAYERS = 10
EMBED_DIM = 1024
NUM_HEADS = 1024  # Car 512 / 32 = 16 entier
FFN_EMBED_DIM = 1024
GAUSSIAN_CHANNELS = 1024
VOCAB_SIZE = 7  # H, C, N, O, S, Cl, [CLS]
PAIR_VOCAB_SIZE = 49  # 7 * 7 types de paires
MAX_ATOMS = 23
DROPOUT = 0

# Affichage des paramètres pour vérification
print("Configuration des paramètres du Uni_Model :")
print(f"NUM_LAYERS: {NUM_LAYERS}")
print(f"EMBED_DIM: {EMBED_DIM}")
print(f"NUM_HEADS: {NUM_HEADS}")
print(f"FFN_EMBED_DIM: {FFN_EMBED_DIM}")
print(f"GAUSSIAN_CHANNELS: {GAUSSIAN_CHANNELS}")
print(f"VOCAB_SIZE: {VOCAB_SIZE}")
print(f"PAIR_VOCAB_SIZE: {PAIR_VOCAB_SIZE}")
print(f"MAX_ATOMS: {MAX_ATOMS}")
print(f"DROPOUT: {DROPOUT}")

# Instanciation du modèle
import torch
Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {Device}")

Model = Uni_Model(
    num_layers=NUM_LAYERS,
    embed_dim=EMBED_DIM,
    num_heads=NUM_HEADS,
    ffn_embed_dim=FFN_EMBED_DIM,
    gaussian_channels=GAUSSIAN_CHANNELS,
    vocab_size=VOCAB_SIZE,
    pair_vocab_size=PAIR_VOCAB_SIZE,
    max_atoms=MAX_ATOMS,
    dropout=DROPOUT
)

Model.to(Device)
print("Modèle Uni_Model instancié avec succès.")

Configuration des paramètres du Uni_Model :
NUM_LAYERS: 10
EMBED_DIM: 1024
NUM_HEADS: 1024
FFN_EMBED_DIM: 1024
GAUSSIAN_CHANNELS: 1024
VOCAB_SIZE: 7
PAIR_VOCAB_SIZE: 49
MAX_ATOMS: 23
DROPOUT: 0
Using device: cuda
Modèle Uni_Model instancié avec succès.


In [8]:
PATH_SAVING_MODEL = '../../Saved_Models/Uni_Model/Uni_Model.pth'

os.makedirs(os.path.dirname(PATH_SAVING_MODEL), exist_ok=True)
# Model = torch.load(PATH_SAVING_MODEL)   

In [9]:
# NB parameters
Nb_Parameters = sum(p.numel() for p in Model.parameters() if p.requires_grad)
print(f"Number of parameters in the model: {Nb_Parameters}")

Number of parameters in the model: 63050856


In [10]:
DataLoader_Train,DataLoader_Val = Create_DataLoader_Uni_Model(List_Data_Train, Batch_Size=8, Shuffle=True, Num_Workers=0, Test_Size=0.2, Return_Energies=True, Transform=None, Max_Atoms=23)

DataLoader_Test = Create_DataLoader_Uni_Model(List_Data_Test, Batch_Size=8, Shuffle=False, Num_Workers=0, Return_Energies=False, Transform=None, Max_Atoms=23)

In [11]:
import torch.optim as optim
import torch.nn as nn

In [12]:
# Empty cache of the GPU
torch.cuda.empty_cache()

Optimizer = torch.optim.Adam(Model.parameters(), lr=8e-4)
# Define the loss function
# Criterion is already defined
Criterion = nn.MSELoss()

# Scheduler: réduit le LR si la loss de validation ne s'améliore pas
Scheduler = torch.optim.lr_scheduler.StepLR(
    Optimizer, step_size=30, gamma=0.5, verbose=True
)

Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

List_Train_Losses_Per_Epochs, List_Test_Losses_Per_Epochs, List_Train_Losses_Per_Batches, List_Test_Losses_Per_Batches = Train(
    Model,
    DataLoader_Train,
    DataLoader_Val,
    Optimizer,
    Criterion,
    Num_Epochs=220,
    Device=Device,
    Scheduler=Scheduler,
    Save_Path=PATH_SAVING_MODEL,
    Best_Val_Loss=float('inf'),
    Coord_Loss_Weight=0.5,
    Energy_Loss_Weight=5.0
)

Plot_Losses(
    List_Train_Losses_Per_Epochs,
    List_Test_Losses_Per_Epochs,
    List_Train_Losses_Per_Batches,
    List_Test_Losses_Per_Batches,
    Save=True,
    Save_Path='losses_plot.png'
)


/home/ayoubchoukri/anaconda3/envs/Personal/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epochs:   0%|          | 0/220 [00:00<?, ?it/s]

Batches:   0%|          | 0/660 [00:00<?, ?it/s]

Batches:   0%|          | 0/165 [00:00<?, ?it/s]

New best validation loss: 4633.009, model saved to ../../Saved_Models/Uni_Model/Uni_Model.pth
Epoch: 1/220 Train Loss: 485.016 Val Loss: 4633.009 Learning Rate: 0.000800


...

Batches:   0%|          | 0/660 [00:00<?, ?it/s]

Batches:   0%|          | 0/165 [00:00<?, ?it/s]

Epoch: 163/220 Train Loss: 12.505 Val Loss: 13.106 Learning Rate: 0.000025


Batches:   0%|          | 0/660 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
.
os.makedirs('./Saved_Models/Transformer', exist_ok=False)

In [ ]:
PATH_SAVING_MODEL = '../Saved_Models/Transformers/V2/Transformer_Atom_Energy_Prediction.pth'
torch.save(Model, PATH_SAVING_MODEL)
print(f"Model saved to {PATH_SAVING_MODEL}")

In [ ]:
# Load weights 

Model_Imported = torch.load(PATH_SAVING_MODEL)

In [ ]:
Model_Imported

In [ ]:
Dataset_Test[0]

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

def Compute_Predictions(Model, Dataset_Test, Device, batch_size=32):
    """
    Computes predictions on the test dataset using the Transformer model.

    Args:
        Model: The Transformer model.
        Dataset_Test: MoleculeDataset_Transformer instance for the test set.
        Device: torch.device (e.g., 'cuda:0' or 'cpu').
        batch_size: Batch size for processing (default: 32).

    Returns:
        Ids: List of molecule identifiers.
        Predictions: List of predicted energy values (scalars).
    """
    Model.eval()  # Set the model to evaluation mode
    Model.to(Device)

    # Create DataLoader for efficient batch processing
    test_loader = DataLoader(Dataset_Test, batch_size=batch_size, shuffle=False)

    Predictions = []
    Ids = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Computing Predictions"):
            Ids_batch, symbols, positions, mask = batch  # Ignore energy
            symbols = symbols.to(Device)
            positions = positions.to(Device)
            mask = mask.to(Device)

            # Forward pass
            outputs = Model(symbols, positions, mask=mask)  # Shape: (batch_size, 1)
            outputs = outputs.squeeze().cpu().tolist()  # Convert to list of scalars

            # Extend lists
            Ids.extend(Ids_batch)
            if isinstance(outputs, float):  # Single prediction case
                Predictions.append(outputs)
            else:  # Batch of predictions
                Predictions.extend(outputs)


    # transform ids to list of ints
    Ids = [int(id) for id in Ids]
    print(Ids)
    return Ids, Predictions

In [ ]:
Ids, Predictions = Compute_Predictions(Model, Dataset_Test, Device)

In [ ]:
Predictions

In [ ]:
# create a csv file with the predictions

import pandas as pd
def Save_Predictions_To_CSV(Ids, Predictions, Output_CSV_Path):
    df = pd.DataFrame({
        'id': Ids,
        'energy': Predictions
    })
        # sort the dataframe by id
    df.sort_values(by='id', inplace=True)
    df.to_csv(Output_CSV_Path, index=False)


    print(f"Predictions saved to {Output_CSV_Path}")

Save_Predictions_To_CSV(Ids, Predictions, 'predictions.csv')

In [ ]:
import numpy as np




def test_rotation_invariance():
    # Crée une "molécule" avec 3 atomes
    original_positions = np.array([
        [0.0, 0.0, 0.0],
        [5.0, 0.0, 0.0],
        [0.0, 1.0, 0.0]
    ])

    transformed_positions = random_translate_and_rotate(original_positions, max_translation=5.0)

    print("Original:\n", original_positions)
    print("Transformé:\n", transformed_positions)

    # Vérification des distances inter-atomiques (doivent rester les mêmes)
    def pairwise_distances(positions):
        n = len(positions)
        distances = []
        for i in range(n):
            for j in range(i + 1, n):
                d = np.linalg.norm(positions[i] - positions[j])
                distances.append(d)
        return np.array(distances)

    d_orig = pairwise_distances(original_positions)
    d_trans = pairwise_distances(transformed_positions)

    print("\nDistances originales:", d_orig)
    print("Distances transformées:", d_trans)

    if np.allclose(d_orig, d_trans, atol=1e-5):
        print("\n✅ La rotation conserve les distances : test réussi.")
    else:
        print("\n❌ Les distances ont changé : problème avec la rotation.")


    # Plot les positions originales et transformées
    import matplotlib.pyplot as plt
    fig = plt.figure(figsize=(10, 5))
    ax = fig.add_subplot(121, projection='3d')
    ax.scatter(original_positions[:, 0], original_positions[:, 1], original_positions[:, 2], color='blue', label='Original')
    ax.set_title('Positions Originales')
    ax = fig.add_subplot(122, projection='3d')
    ax.scatter(transformed_positions[:, 0], transformed_positions[:, 1], transformed_positions[:, 2], color='red', label='Transformé')
    ax.set_title('Positions Transformées')
    plt.show()
    

# Lancer le test
test_rotation_invariance()
